In [ ]:
A0 = 0x67452301;
B0 = 0xefcdab89;
C0 = 0x98badcfe;
D0 = 0x10325476;

def F(x, y, z):
    return (x & y) | ((~x) & z)

def G(x, y, z):
    return (x & y) | (x & z) | (y & z)

def H(x, y, z):
    return x ^ y ^ z

def left_rotate(x, n):
    """Left rotate a 32-bit integer x by n bits."""
    return ((x << n) | (x >> (32 - n))) & 0xFFFFFFFF

def preprocess(message_bytes):

    m = bytearray(message_bytes)
    bit_length = len(m) * 8

    # Append 0x80 (10000000 in binary)
    m.append(0x80)

    # Compute number of zero bytes to append
    padding_len = (56 - (len(m) % 64)) % 64
    m.extend([0x00] * padding_len)

    # Append original bit length as a 64-bit little-endian integer
    m.extend(bit_length.to_bytes(8, byteorder='little'))

    return m


def md4_hash(message_str):
    """
    Computes the MD4 hash of the given string.
    Returns a 32-character hex digest.
    """
    # Convert string to bytes, then pad the message
    message = message_str.encode("utf-8")
    padded_message = preprocess(message)
    # print(padded_message)
    # Initialize MD4 state
    A = A0
    B = B0
    C = C0
    D = D0
    print(len(padded_message))
    # Process the message in successive 512-bit (64-byte) chunks
    for i in range(0, len(padded_message), 64):
        block = padded_message[i:i+64]
        print("block ",block)
        # Break block into 16 little-endian 32-bit words
        X = [int.from_bytes(block[j:j+4], byteorder='little') for j in range(0, 64, 4)]
        print("X ",X)
        # Save state
        AA, BB, CC, DD = A, B, C, D
        # Round 1
        # Each step: [abcd k s] means a = (a + F(b, c, d) + X[k]) <<< s
        A = left_rotate((A + F(B, C, D) + X[ 0]) & 0xFFFFFFFF,  3)
        D = left_rotate((D + F(A, B, C) + X[ 1]) & 0xFFFFFFFF,  7)
        C = left_rotate((C + F(D, A, B) + X[ 2]) & 0xFFFFFFFF, 11)
        B = left_rotate((B + F(C, D, A) + X[ 3]) & 0xFFFFFFFF, 19)

        A = left_rotate((A + F(B, C, D) + X[ 4]) & 0xFFFFFFFF,  3)
        D = left_rotate((D + F(A, B, C) + X[ 5]) & 0xFFFFFFFF,  7)
        C = left_rotate((C + F(D, A, B) + X[ 6]) & 0xFFFFFFFF, 11)
        B = left_rotate((B + F(C, D, A) + X[ 7]) & 0xFFFFFFFF, 19)

        A = left_rotate((A + F(B, C, D) + X[ 8]) & 0xFFFFFFFF,  3)
        D = left_rotate((D + F(A, B, C) + X[ 9]) & 0xFFFFFFFF,  7)
        C = left_rotate((C + F(D, A, B) + X[10]) & 0xFFFFFFFF, 11)
        B = left_rotate((B + F(C, D, A) + X[11]) & 0xFFFFFFFF, 19)

        A = left_rotate((A + F(B, C, D) + X[12]) & 0xFFFFFFFF,  3)
        D = left_rotate((D + F(A, B, C) + X[13]) & 0xFFFFFFFF,  7)
        C = left_rotate((C + F(D, A, B) + X[14]) & 0xFFFFFFFF, 11)
        B = left_rotate((B + F(C, D, A) + X[15]) & 0xFFFFFFFF, 19)

        # Round 2
        # Each step: a = (a + G(b, c, d) + X[k] + 0x5A827999) <<< s
        A = left_rotate((A + G(B, C, D) + X[ 0] + 0x5A827999) & 0xFFFFFFFF,  3)
        D = left_rotate((D + G(A, B, C) + X[ 4] + 0x5A827999) & 0xFFFFFFFF,  5)
        C = left_rotate((C + G(D, A, B) + X[ 8] + 0x5A827999) & 0xFFFFFFFF,  9)
        B = left_rotate((B + G(C, D, A) + X[12] + 0x5A827999) & 0xFFFFFFFF, 13)

        A = left_rotate((A + G(B, C, D) + X[ 1] + 0x5A827999) & 0xFFFFFFFF,  3)
        D = left_rotate((D + G(A, B, C) + X[ 5] + 0x5A827999) & 0xFFFFFFFF,  5)
        C = left_rotate((C + G(D, A, B) + X[ 9] + 0x5A827999) & 0xFFFFFFFF,  9)
        B = left_rotate((B + G(C, D, A) + X[13] + 0x5A827999) & 0xFFFFFFFF, 13)

        A = left_rotate((A + G(B, C, D) + X[ 2] + 0x5A827999) & 0xFFFFFFFF,  3)
        D = left_rotate((D + G(A, B, C) + X[ 6] + 0x5A827999) & 0xFFFFFFFF,  5)
        C = left_rotate((C + G(D, A, B) + X[10] + 0x5A827999) & 0xFFFFFFFF,  9)
        B = left_rotate((B + G(C, D, A) + X[14] + 0x5A827999) & 0xFFFFFFFF, 13)

        A = left_rotate((A + G(B, C, D) + X[ 3] + 0x5A827999) & 0xFFFFFFFF,  3)
        D = left_rotate((D + G(A, B, C) + X[ 7] + 0x5A827999) & 0xFFFFFFFF,  5)
        C = left_rotate((C + G(D, A, B) + X[11] + 0x5A827999) & 0xFFFFFFFF,  9)
        B = left_rotate((B + G(C, D, A) + X[15] + 0x5A827999) & 0xFFFFFFFF, 13)

        # Round 3 
        # Each step: a = (a + H(b, c, d) + X[k] + 0x6ED9EBA1) <<< s
        A = left_rotate((A + H(B, C, D) + X[ 0] + 0x6ED9EBA1) & 0xFFFFFFFF,  3)
        D = left_rotate((D + H(A, B, C) + X[ 8] + 0x6ED9EBA1) & 0xFFFFFFFF,  9)
        C = left_rotate((C + H(D, A, B) + X[ 4] + 0x6ED9EBA1) & 0xFFFFFFFF, 11)
        B = left_rotate((B + H(C, D, A) + X[12] + 0x6ED9EBA1) & 0xFFFFFFFF, 15)

        A = left_rotate((A + H(B, C, D) + X[ 2] + 0x6ED9EBA1) & 0xFFFFFFFF,  3)
        D = left_rotate((D + H(A, B, C) + X[10] + 0x6ED9EBA1) & 0xFFFFFFFF,  9)
        C = left_rotate((C + H(D, A, B) + X[ 6] + 0x6ED9EBA1) & 0xFFFFFFFF, 11)
        B = left_rotate((B + H(C, D, A) + X[14] + 0x6ED9EBA1) & 0xFFFFFFFF, 15)

        A = left_rotate((A + H(B, C, D) + X[ 1] + 0x6ED9EBA1) & 0xFFFFFFFF,  3)
        D = left_rotate((D + H(A, B, C) + X[ 9] + 0x6ED9EBA1) & 0xFFFFFFFF,  9)
        C = left_rotate((C + H(D, A, B) + X[ 5] + 0x6ED9EBA1) & 0xFFFFFFFF, 11)
        B = left_rotate((B + H(C, D, A) + X[13] + 0x6ED9EBA1) & 0xFFFFFFFF, 15)

        A = left_rotate((A + H(B, C, D) + X[ 3] + 0x6ED9EBA1) & 0xFFFFFFFF,  3)
        D = left_rotate((D + H(A, B, C) + X[11] + 0x6ED9EBA1) & 0xFFFFFFFF,  9)
        C = left_rotate((C + H(D, A, B) + X[ 7] + 0x6ED9EBA1) & 0xFFFFFFFF, 11)
        B = left_rotate((B + H(C, D, A) + X[15] + 0x6ED9EBA1) & 0xFFFFFFFF, 15)

        A = (A + AA) & 0xFFFFFFFF
        B = (B + BB) & 0xFFFFFFFF
        C = (C + CC) & 0xFFFFFFFF
        D = (D + DD) & 0xFFFFFFFF

    digest = (A.to_bytes(4, byteorder="little") +
              B.to_bytes(4, byteorder="little") +
              C.to_bytes(4, byteorder="little") +
              D.to_bytes(4, byteorder="little")).hex()
    return digest

test_vectors = [
    ("", "31d6cfe0d16ae931b73c59d7e0c089c0"),
    ("a", "bde52cb31de33e46245e05fbdbd6fb24"),
    ("abc", "a448017aaf21d8525fc10ae87aa6729d"),
    ("message digest", "d9130a8164549fe818874806e1c7014b"),
    ("abcdefghijklmnopqrstuvwxyz", "d79e1c308aa5bbcdeea8ed63df412da9"),
    ("ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz0123456789",
     "043f8582f241db351ce627e153e7f0e4")
]

print("Testing MD4 implementation:")
for text, expected in test_vectors:
    result = md4_hash(text)
    print(f"Input: '{text}'")
    print(f"Expected: {expected}")
    print(f"Result:   {result}")
    print(f"Match:    {result == expected}\n")
i=input("enter string: ")
print(md4_hash(i))

Testing MD4 implementation:
64
block  bytearray(b'\x80\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00')
X  [128, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Input: ''
Expected: 31d6cfe0d16ae931b73c59d7e0c089c0
Result:   31d6cfe0d16ae931b73c59d7e0c089c0
Match:    True

64
block  bytearray(b'a\x80\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x08\x00\x00\x00\x00\x00\x00\x00')
X  [32865, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 8, 0]
Input: 'a'
Expected: bde52cb31de33e46245e05fbdbd6fb24
Result:   bde52cb31de33e46245e05fbdbd6fb24
Match:    True

64
block  bytearray(b'abc\x80\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\